# CODE FOR AUTOMATION OF DATA COLLECTION FOR BENCH TEST AND DETECTOR
## By Damien Bowen, LBNL

### Initialization (packages, connecting to instrumentation)

In [1]:
## packages
import shutil
import pyvisa
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import struct 
import os
from datetime import datetime
import h5py

##### Electronics have had their ip address configured as given below

In [2]:
## initializing resource manager for the electronics
rm = pyvisa.ResourceManager('@py')

In [15]:
## connecting to the oscilloscope (essential for both detector tests and bench tests)
scope = rm.open_resource('TCPIP0::169.254.7.109')
print(scope.query('*IDN?')) ## prints make and model of oscilloscope to verify connection

TEKTRONIX,MSO44B,SGVJ010162,CF:91.1CT FV:2.6.38.1348



In [ ]:
## connecting to pulse generator
pulser = rm.open_resource('TCPIP0::169.254.7.110')
print(pulser.query('*IDN?'))

In [ ]:
## connecting to power supply; finnicky compared to other devices, needs extra work
ip_address = '169.254.7.111'
resource_string = f'TCPIP0::{ip_address}::5025::SOCKET'
supply = rm.open_resource(resource_string)
print(supply.query('*IDN?'))

# Definition of functions, categorized as best as possible below

### For interacting with the electronics:

In [16]:
## configures power supply to desired format
def configure_power_supply():
    channel = 3
    voltage = 2.8
    current_limit = 50
    
    # Select the channel
    supply.write(f'INST:NSEL {channel}')
    
    # Set voltage
    supply.write(f'VOLT {voltage}')
    
    # Set current limit
    supply.write(f'CURR {current_limit}')
    
    # Turn on the output
    supply.write('OUTP ON')
    
    # Read back current
    current = float(supply.query('MEAS:CURR?'))
    
    # Check if the current is within the desired range
    if .005 <= current <= .010:
        print(f'Current is within range: {current} A')
    else:
        print(f'Current is out of range: {current} A')


In [17]:
## changes voltage of pulser
def pulser_v(voltage): ## parameter in units of volts
    return pulser.write(f'C1:BSWV AMP, {voltage}') 

In [18]:
## changes vertical (voltage) scaling of oscilloscope, needed for optimizing resolution at diff voltages for bench test
def set_vert_scale(channel, scale):
    scope.write(f':CH{channel}:SCALE {scale}')

In [19]:
## changes time scale of oscilloscope
def set_horizontal_scale(scale):
    scope.write(f':HOR:MAIN:SCALE {scale}')

In [20]:
## gets measurement from any of the measurement channels on oscilloscope
def get_measurement(measurement_channel):
    # Query the value of the specified measurement channel
    measurement = scope.query(f':MEASU:MEAS{measurement_channel}:VALue?')
    return float(measurement)

In [21]:
## make sure to adjust which channel you want to use for the trigger
def set_trigger(trigger_level):
    # Construct the SCPI command to set the trigger source
    scope.write('TRIGger:A:EDGE:SOURce CH4') ## makes sure the trigger is set to channel 4
        
    # Construct the SCPI command to set the trigger level
    scope.write(f'TRIGger:A:LEVel:CH4 {trigger_level}') ## set trigger as low as possible 

In [22]:
# getting waveform data from oscilloscope for a given channel
def get_waveform_data(channel):
    # Select the channel
    scope.write(f'DATA:SOURCE CH{channel}')
    scope.write(f'DATA:WIDTH 2')
    
    raw_data = scope.query_binary_values('CURVE?', datatype = 'b', container=np.array)

# Horizontal scale settings to reconstruct time data
    x_increment = float(scope.query('WFMPRE:XINCR?'))
    x_zero = float(scope.query('WFMPRE:XZERO?'))
    x_offset = float(scope.query('WFMPRE:PT_OFF?'))
    # print(x_offset, x_zero)

# Get vertical scale settings to reconstruct voltage data
    y_multiplier = float(scope.query('WFMPRE:YMULT?'))
    y_offset = float(scope.query('WFMPRE:YOFF?'))
    y_zero = float(scope.query('WFMPRE:YZERO?'))

# Convert raw data to numpy array
    data = np.frombuffer(raw_data, dtype=np.int16)  # Adjust dtype if needed

# Reconstruct time and voltage data
    time_data = x_zero + (np.arange(len(data)) - x_offset) * x_increment
    voltage_data = y_zero + (data - y_offset)*y_multiplier

    return time_data, voltage_data

In [23]:
## corrects the vertical scaling of the oscilloscope based on the peak voltage. 
## this function is used in combination with get_measurement() to get the peak voltage through the device 
## being tested.
def correct_y_scaling(wave_amplitude):

    ## takes in amplitude of the ASIC (a float) and adjusts y divisions of oscilloscope as required
    if 1.6 < wave_amplitude <= 4:
        set_trigger(0.1)
        set_vert_scale(1, 0.5)
        set_vert_scale(4, 0.5)
    elif 0.8 <= wave_amplitude <= 1.6:
        set_vert_scale(1, 0.2)  # 200 mV/div for both ASIC readout, pulser
        set_vert_scale(4, 0.2)
    elif 0.4 <= wave_amplitude < 0.8:
        set_vert_scale(1, 0.1)  # 100 mV/div for ASIC, still 200 for pulser
    elif 0.18 <= wave_amplitude < 0.4:
        set_trigger(0.05)
        set_vert_scale(1, 0.05)  # 50 mV/div on ASIC readout
        set_vert_scale(4, 0.1)   # 100 mV/div for pulser
    elif 0.08 <= wave_amplitude < 0.18:
        set_trigger(0.05)
        set_vert_scale(1, 0.02)  # 20 mV/div
        set_vert_scale(4, 0.05)  # 50 mV/div for pulser
    elif 0.04 <= wave_amplitude < 0.08:
        set_trigger(0.02)
        set_vert_scale(1, 0.01)  # 10 mV/div
        set_vert_scale(4, 0.02)  # 20 mV/div for pulser
    elif 0.015 <= wave_amplitude < 0.04:
        set_trigger(0.01)
        set_vert_scale(1, 0.005)  # 5 mV / div
        set_vert_scale(4, 0.01)   # 10 mV/div for pulser
    elif .008 <= wave_amplitude < 0.015:
        set_trigger(0.004)
        set_vert_scale(1, 0.002)
    elif 0.0 <= wave_amplitude < .008:
        set_vert_scale(1, 0.001)


# Saving data to different file types. Currently, .csv and .hdf5 are available (.hdf5 is significantly faster)

##### code for creation of folders:

In [24]:
## creates folder for .csv files to be inserted into. The folder is automatically created in this directory
## in the folder I have called ./Data.
## GIVE THE FOLDER A NAME IN THE ALGORITHMS
def create_folder(base_dir='./Data', name = None, run_type = None, n_runs = None, subfolder = None):
    # If a subfolder is specified, use it directly without generating a new folder name
    if subfolder:
        folder_path = os.path.join(base_dir, subfolder)
    else:
        # Generate a folder name if 'name' is not provided and 'subfolder' is not specified
        if name is None:
            name = f'ASIC_data_{int(time.time())}'
        else:
            current_date = datetime.datetime.now().strftime('%Y%m%d')
            name = f'{name}_{current_date}'
        folder_path = os.path.join(base_dir, name)
    
    # Create the folder (or subfolder) path
    os.makedirs(folder_path, exist_ok=True)
    # print(f'Folder created: {folder_path}')
    
    ## create a .csv file with important metadata for main data collection folders
    if not subfolder:
        info_filename = 'run_info.csv'
        info_filepath = os.path.join(folder_path, info_filename)
        current_date = datetime.datetime.now().strftime('%Y-%m-%d')
        current_time = datetime.datetime.now().strftime('%H:%M:%S')

        with open(info_filepath, 'w', newline='') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(['Oscilloscope: TEKTRONIX MSO44B', 'Pulser: TELEDYNE T3AFG500'])
            csvwriter.writerow([f'Date: {current_date}', f'Time: {current_time}'])
            csvwriter.writerow(['Time units: microseconds', 'Voltage units: Volts'])


            if run_type:
                csvwriter.writerow([f'Run type: {run_type}'])

            if n_runs:
                csvwriter.writerow([f'# of iterations at each voltage: {n_runs}'])
    
    return folder_path

### .csv code:

In [25]:
## saves ASIC time data, voltage data, pulser voltage data to .csv file as well as other parameters
def save_to_csv(path, time_data, device_voltage_data, pulser_voltage_data, pulser_voltage, asic_voltage, counter = None):
    
    filename = f'ASIC_A{asic_voltage:.3f}V_P{pulser_voltage:.3f}V_{counter:03d}.csv'.replace('.','p',2)  
    fullpath = os.path.join(path, filename)
    
    with open(fullpath, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        
        csvwriter.writerow([f'Pulser Voltage: {pulser_voltage:.3f}V', f'ASIC Peak Voltage: {asic_voltage:.3f}V' ])
        csvwriter.writerow(['Time (s)', 'ASIC Voltage (V)','Pulser Voltage (V)'])  # Header row
        for t, v, q in zip(time_data, device_voltage_data, pulser_voltage_data):
            csvwriter.writerow([t, v, q])
   

In [26]:
## developed for usage with the detector specifically. I am keeping it separate for now
## because I am not sure it will work with the bench test algorithm.

def save_to_csv_detector(path, time_data, device_voltage_data, pulser_voltage_data = None, pulser_voltage=None, asic_voltage=None, counter=None):
    # Build the filename based on available data
    filename_parts = []
    
    filename_parts.append("ASIC_data")
    
    # Only append ASIC and Pulser information if they are not None
    if asic_voltage is not None:
        filename_parts.append(f"ASIC_{asic_voltage:.3f}V")
        
    if pulser_voltage is not None:
        filename_parts.append(f"P{pulser_voltage:.3f}V")
    
    
    if counter is not None:
        filename_parts.append(f"{counter:03d}")
    
    # If no voltages are provided, the filename will only contain date, time, and counter
    if not filename_parts:
        filename = f"ASIC_data_{timestamp}_{counter:03d}.csv" if counter is not None else f"ASIC_data_{timestamp}.csv"
    else:
        # Create the filename from parts and replace '.' with 'p' for formatting
        filename = '_'.join(filename_parts).replace('.', 'p', 2) + ".csv"
    
    # Create the full file path
    fullpath = os.path.join(path, filename)
    
    # Open the CSV file for writing
    with open(fullpath, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)

        # Write header based on available voltage data
        pulser_info = f'Pulser Voltage: {pulser_voltage:.3f}V' if pulser_voltage is not None else 'Pulser Voltage: None'
        asic_info = f'ASIC Peak Voltage: {asic_voltage:.3f}V' if asic_voltage is not None else 'ASIC Peak Voltage: None'
        
        # First row: Summary info
        csvwriter.writerow([pulser_info, asic_info])
        
        # Second row: Column headers
        csvwriter.writerow(['Time (s)', 'ASIC Voltage (V)', 'Pulser Voltage (V)'])
        
        # Write data rows; use placeholder for missing pulser voltage data
        if pulser_voltage_data is None:
            # Fill missing pulser voltage data with 'None'
            for t, v in zip(time_data, device_voltage_data):
                csvwriter.writerow([t, v, 'None'])
        else:
            # Normal case with pulser voltage data
            for t, v, q in zip(time_data, device_voltage_data, pulser_voltage_data):
                csvwriter.writerow([t, v, q])

### for .hdf5 files:

In [27]:
## data to write a waveform into an hdf5 file
def write_waveform_to_hdf5(hdf_file, group_name, time_data, voltage_data, run_num):
    """
    Writes waveform data to HDF5 file under the specified group.

    Parameters:
    hdf_file (h5py.File): The open HDF5 file object.
    group_name (str): The name of the group (e.g., voltage level).
    time_data (list or np.array): The time data points.
    voltage_data (list or np.array): The corresponding voltage data points.
    run_num (int): The current run number to name the dataset.
    """
    # Combine the time and voltage data into a 2D array
    waveform_data = np.vstack((time_data, voltage_data)).T
    
    # Create a dataset for the waveform in the specified group
    dataset_name = f"waveform_{run_num:05d}"
    
    # Write the dataset with compression
    hdf_file[group_name].create_dataset(dataset_name, data=waveform_data, compression="gzip")
    
    """Optional print statement for debugging and tracking."""
    # print(f"Waveform {run_num + 1} saved to group {group_name}.")

In [28]:
## Function to automatically upload data to google drive

def save_hdf5_to_drive(hdf_file_path):
    
    ## constructing the path to the file you want to upload to the google drive
    source_file_path = hdf_file_path
    
    ## constructing path to the google drive
    google_drive_path = google_drive_path = r"/Users/ilvuoto/Library/CloudStorage/GoogleDrive-dvbowen@lbl.gov/My Drive/ASIC_data_drive"

    ## moving the source file to the google drive
    shutil.move(source_file_path, google_drive_path)
    
    print(f"Moved data file to {google_drive_path}.")
    

In [29]:
### function to write important metadata into each file

def write_hdf5_metadata(hdf_file, run_type = None):    
    ### METADATA
    hdf_file.attrs['DEVICE_NAME'] = 'LBNL GFET distributed CSA'
    hdf_file.attrs['DEVICE_VOLTAGE'] = '1.0 V'
    hdf_file.attrs['V_MID'] = '0.8 V'
    hdf_file.attrs['POWER_SUPPLY_PARAMETERS'] = ['5V, 3mA and -5V, 3mA']
    hdf_file.attrs['SCOPE_PARAMETERS'] = ['AC termination, 75 Ohms']
    hdf_file.attrs['HPGe_LBNL_PPC_BIAS_VOLTAGE'] = ['2 kV']
    
    if run_type:
        hdf_file.attrs['RUN_TYPE'] = run_type

## DATA COLLECTION FUNCTIONS

##### for .csv and folders method:

In [32]:
## Used for bench tests. 
# PARAMETERS:
# voltage_array: array of voltages to set pulser to for testing
# num_runs: number of runs at each voltage in the array
# folder_title: GIVE THE FOLDER A TITLE like the date of the acquisition, device being tested, etc.
def collect_data(voltage_array, num_runs, folder_title = None):
    ## create folder title for this run with given title, type of run in this case iterates over an array 
    ## at each voltage multiple times. 
    folder_path = create_folder(name = folder_title, 
                                run_type = f'Array of voltages, each iterated over', 
                                n_runs = num_runs)
    
    ## iterating over each voltage in the voltage array
    for volt in voltage_array:
        
        subfolder_name = f'ASIC_PulserVoltage_{volt:.3f}V'.replace('.','p', 1)
        subfolder_path = create_folder(base_dir = folder_path, subfolder = subfolder_name)
        
        counter = 0      ## to save multiple files at same voltage
        pulser_v(volt)   ## sets the voltage on the pulser
        time.sleep(.01)    ## adds delay for waveform to stabilize before data collection
        
        
        asic_amplitude = get_measurement(3) ## gets ASIC max amplitude so we can adjust scaling
        
        adjust_y_divisions(asic_amplitude)  ## adjusts y scaling for max resolution using asic amplitude
        
        ## collects n waveforms at each voltage
        for n in range(num_runs):
            time_wave, voltage_wave               = get_waveform_data(1) ## gets ASIC waveform data out of channel 1
            time_wave_pulser, voltage_wave_pulser = get_waveform_data(4) ## gets pulser voltages 
            save_to_csv(subfolder_path, time_wave, voltage_wave, voltage_wave_pulser, volt, asic_amplitude)
            counter += 1
    
    

In [33]:
## FOR USE WITH DETECTOR

# PARAMETERS:
# max_events: number of events saved to .csv file before algorithm terminates
def data_scan(max_events = 1000):
    
    folder_path = create_folder(base_dir = "./Data", name = "ASIC_detector_run_test3")
    count = 0
    print("Scanning begins..")
    
    while True:
    # Poll the oscilloscope's trigger status
        scope.write('TRIGger:STATE?')
        trigger_status = scope.read().strip()
    
    # Check if the trigger status indicates an event
        if trigger_status == 'TRIGGER':  # This indicates an event occurred
            print("Event detected!")
        
            time_wave, voltage_wave               = get_waveform_data(1) ## gets ASIC waveform data out of channel 1
            time_wave_pulser, voltage_wave_pulser = get_waveform_data(2) ## gets pulser voltages 
            save_to_csv_detector(path = folder_path, time_data = time_wave, device_voltage_data = voltage_wave, pulser_voltage_data = voltage_wave_pulser, counter = count)
        
            ## counter to keep track of how many events have been saved
            count += 1
        
            if count >= max_events:
                break

##### hdf5 file method:

In [37]:
def collect_data_hdf5(voltage_array, num_runs, file_name = None, run_title=None):
    """
    Collects waveform data over multiple runs at different voltages and saves them in an HDF5 file.
    
    Parameters:
    voltage_array (list): Array of voltages to test.
    num_runs (int): Number of runs per voltage.
    file_name (str): Name of the HDF5 file.
    run_title (str): Optional title for the run, used to name the group in the HDF5 file.
                     If None, default will be 'ASIC_bench_test_{timestamp}'.
    """
    # Ensure the file is created in the ./Data directory by default
    base_dir = './Data'
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)  # Create the directory if it doesn't exist

    # Build the full path by appending the file name to the ./Data directory
    hdf5_file_path = os.path.join(base_dir, file_name)

    if file_name is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"ASIC_BenchTest_{timestamp}"
    
    
    # Generate a default run title with a timestamp if run_title is not provided
    if run_title is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        run_title = f"ASIC_bench_test_{timestamp}"

    # Open or create the HDF5 file
    with h5py.File(hdf5_file_path, 'a') as hdf_file:
        
        # Write metadata for run type, device parameters, etc
        write_hdf5_metadata(hdf_file, run_type = 'Bench test')
        
        # Create a group for this run using the run_title
        run_group = hdf_file.require_group(run_title)

        # Iterating over each voltage in the voltage array
        for volt in voltage_array:
            # Create a subgroup for the current voltage
            voltage_group_name = f"PulserVoltage_{volt:.3f}V".replace('.', 'p', 1)
            voltage_group = run_group.require_group(voltage_group_name)

            # Set the voltage on the pulser
            pulser_v(volt)
            time.sleep(0.3)  # Delay for voltage stabilization

            # Get ASIC max amplitude to adjust scaling
            asic_amplitude = get_measurement(3)
            
            # Adjust y scaling for maximum resolution using ASIC amplitude
            adjust_y_divisions(asic_amplitude)

            # Collect num_runs waveforms at this voltage
            for run_num in range(num_runs):
                # Get the waveform data
                time_wave, voltage_wave = get_waveform_data(1)
                
                # Save the waveform to HDF5 file
                write_waveform_to_hdf5(hdf_file, voltage_group_name, time_wave, voltage_wave, run_num)

                # Print progress
                print(f"Run {run_num + 1}/{num_runs} complete for voltage {volt:.3f}V.")
    
    # print(f"Data collection complete. Results saved to {hdf5_file_path}."
    
    save_hdf5_to_drive(hdf5_file_path)
    
    print("Data moved to ASIC_data_drive in the Google Drive.")

In [40]:
## FOR USE WITH DETECTOR

# PARAMETERS same as .csv version.
def data_scan_hdf5(max_events=1000):
        
    # Open HDF5 file for storing the waveforms
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    hdf5_file_path = f'./Data/ASIC_detector_run_{timestamp}.hdf5'  # You can modify the name or path as needed
    hdf_file = h5py.File(hdf5_file_path, 'a')    # Open HDF5 in append mode
    
    write_hdf5_metadata(hdf_file, run_type = 'Detector test')
    
    run_title = f"ASIC_detector_run_{timestamp}"
    run_group = hdf_file.require_group(run_title) ## creating a group for this run so hierarchy is the same
    
    group_name = run_group.name
    
    count = 0
    print("Scanning begins..")
    
    while True:
        # Poll the oscilloscope's trigger status
        scope.write('TRIGger:STATE?')
        trigger_status = scope.read().strip()
    
        # Check if the trigger status indicates an event
        if trigger_status == 'TRIGGER':  # This indicates an event occurred
            # print("Event detected!")
        
            # Get the waveform data
            time_wave, voltage_wave = get_waveform_data(1)  # ASIC waveform data out of channel 1
            # time_wave_pulser, voltage_wave_pulser = get_waveform_data(2)  # Pulser voltages
            
            # Save waveform to HDF5 file
            write_waveform_to_hdf5(hdf_file, group_name, time_wave, voltage_wave, count)
        
            # Increment counter for tracking how many events have been saved
            count += 1
        
            if count >= max_events:
                break
    
    save_hdf5_to_drive(hdf5_file_path)

# ACTUAL DATA COLLECTION

### initializing machinery:

In [ ]:
## pulser parameters
rise_time = 10e-9 
fall_time = 10e-9  ## 10 ns rise and fall time
pulse_width = .1   ## 100 ms pulse width
pulser_delay = 0.0 ## no delay

set_horizontal_scale(100e-6) ## sets time axis of oscilloscope to 40 microseconds


pulser.write(f':PULS:TRAN:LEAD {10e-9}') 
pulser.write(f':PULS:TRAN:TRA {10e-9}')  ## rise and fall time to 10 nanos
pulser.write(f':PULS:WIDth {.1}')

pulser.write(f':PULS:DEL {0.0}') ## 0 delay 

In [ ]:
## Run the configuration function for power supply
configure_power_supply()

In [ ]:
## INITIALIZING SCOPE
set_trigger(.1)
pulser_v(3.5)
set_vert_scale(4, .5)
set_vert_scale(1, .5)


## STANDARDIZE SCOPE OFFSET SO ITS IDENTICAL ACROSS RUNS
scope.write('CH1:POS -4.00')
scope.write('CH4:POS -4.00')


## SET ACQUISITION MODE TO HIGH RESOLUTION
scope.write(':ACQ:MODE HIRES')

### .csv method: bench test

In [ ]:
### array of voltages to run through, modify as needed
voltage_array   =    [4.5, 4.4, 4.3, 4.2, 4.1, 4.0, 
                      3.9, 3.8, 3.7, 3.6, 3.5, 3.4, 3.3, 3.2, 3.1, 3.0, 
                      2.9, 2.8, 2.7, 2.6, 2.5, 2.4, 2.3, 2.2, 2.1, 2.0,
                      1.9, 1.8, 1.7, 1.6, 1.5, 1.4, 1.3, 1.2, 1.1, 1.0,
                      .9, .8, .7, .6, .5, .4, .3, .2, .1, 
                      .09, .08, .07, .06, .05, .04, .03, .02, .017, .013, .01] 



timestamp = current_time.strftime("%Y/%m/%d")
collect_data(voltage_array, 1000, 'ASIC_data_{timestamp}')

### .csv method: detector test

In [ ]:
data_scan(max_events = 1000)

### .hdf5 method: bench test

In [ ]:
### array of voltages to run through, modify as needed
volt_array   =    [4.5, 4.4, 4.3, 4.2, 4.1, 4.0, 
                   3.9, 3.8, 3.7, 3.6, 3.5, 3.4, 3.3, 3.2, 3.1, 3.0, 
                   2.9, 2.8, 2.7, 2.6, 2.5, 2.4, 2.3, 2.2, 2.1, 2.0,
                   1.9, 1.8, 1.7, 1.6, 1.5, 1.4, 1.3, 1.2, 1.1, 1.0,
                   .9, .8, .7, .6, .5, .4, .3, .2, .1, 
                   .09, .08, .07, .06, .05, .04, .03, .02, .017, .013, .01] 


collect_data_hdf5(voltage_array = volt_array, num_runs = 1000, )

### .hdf5 method: detector test

In [41]:
data_scan_hdf5(max_events = 1000)

Scanning begins..


KeyboardInterrupt: 